# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [17]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Load the CSV (cities.csv)
clean_city_data_path = "../output_data/cities.csv"

clean_city = pd.read_csv(clean_city_data_path)
clean_city

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Fortuna,40.60,-124.16,69.01,77,40,10.29,US,1596146101
1,Qaanaaq,77.48,-69.36,43.05,63,19,5.55,GL,1596145545
2,Esperance,-33.87,121.90,44.60,100,40,5.82,AU,1596145834
3,Cabo San Lucas,22.89,-109.91,89.60,58,5,14.99,MX,1596145767
4,Port Alfred,-33.59,26.89,56.71,85,27,7.63,ZA,1596145765
...,...,...,...,...,...,...,...,...,...
591,Rockhampton,-23.38,150.50,57.20,82,0,4.70,AU,1596146444
592,Aykhal,66.00,111.50,53.60,71,0,4.47,RU,1596145876
593,Lamar,33.67,-95.58,98.60,44,1,16.11,US,1596146445
594,Kendari,-3.94,122.50,72.57,93,92,4.47,ID,1596146445


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
!jupyter nbextension enable --py gmaps
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = clean_city[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
clean_city = clean_city.dropna()
humidity = clean_city["Humidity"].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

fig.add_layer(heat_layer)

fig


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# A max temperature lower than 80 degrees but higher than 70.
best_temp = clean_city.loc[(clean_city["Max Temp"] >70) & (clean_city["Max Temp"] <80)]
best_temp

# Wind speed less than 10 mph.
wind_speed = best_temp.loc[(best_temp["Wind Speed"] <10)]
wind_speed

# Zero cloudiness.
zero_clouds = wind_speed.loc[(wind_speed["Cloudiness"] <=0)]
zero_clouds

# Humidity less than 65.
ideal_weather = zero_clouds.loc[(zero_clouds["Humidity"] <65)]
ideal_weather


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
279,Birjand,32.87,59.22,73.40,20,0,4.70,IR,1596146358
388,Haines Junction,60.75,-137.51,75.40,37,0,8.81,CA,1596145814
504,Wuda,39.50,106.71,75.15,52,0,2.30,CN,1596146423
522,Abu Samrah,35.30,37.18,77.05,48,0,6.91,SY,1596146427
578,Barra,-11.09,-43.14,76.32,42,0,7.27,BR,1596146441
580,Miepherkeim,38.14,41.01,75.20,50,0,6.93,TR,1596146441


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = ideal_weather

hotel_df['Hotel Name'] = ""
hotel_df

/Users/valpopovich/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
279,Birjand,32.87,59.22,73.40,20,0,4.70,IR,1596146358,
388,Haines Junction,60.75,-137.51,75.40,37,0,8.81,CA,1596145814,
504,Wuda,39.50,106.71,75.15,52,0,2.30,CN,1596146423,
522,Abu Samrah,35.30,37.18,77.05,48,0,6.91,SY,1596146427,
578,Barra,-11.09,-43.14,76.32,42,0,7.27,BR,1596146441,
580,Miepherkeim,38.14,41.01,75.20,50,0,6.93,TR,1596146441,


In [20]:
# Find the closest hotel within 5000 meters of each city

target_search = "hotel"
target_radius = 5000
target_type = "lodging"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# Use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"] 

    # Add keyword to params dictionary
    params['location'] = f"{lat},{lng}"

    # Assemble URL and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    print("-" *20)
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response['results']
    # print(results)
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 279: Birjand.
--------------------
Closest hotel is Sepehr Hotel.
------------
Retrieving Results for Index 388: Haines Junction.
--------------------


/Users/valpopovich/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel is Raven's Rest Inn.
------------
Retrieving Results for Index 504: Wuda.
--------------------
Closest hotel is Xinxin Hotel.
------------
Retrieving Results for Index 578: Barra.
--------------------
Closest hotel is Hotel Barra Palace.
------------
Retrieving Results for Index 580: Miepherkeim.
--------------------
Closest hotel is Silvan Grand Otel.
------------


In [21]:
hotel_df = hotel_df[hotel_df["Hotel Name"]!= ""]

In [22]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
279,Birjand,32.87,59.22,73.40,20,0,4.70,IR,1596146358,Sepehr Hotel
388,Haines Junction,60.75,-137.51,75.40,37,0,8.81,CA,1596145814,Raven's Rest Inn
504,Wuda,39.50,106.71,75.15,52,0,2.30,CN,1596146423,Xinxin Hotel
578,Barra,-11.09,-43.14,76.32,42,0,7.27,BR,1596146441,Hotel Barra Palace
580,Miepherkeim,38.14,41.01,75.20,50,0,6.93,TR,1596146441,Silvan Grand Otel


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
!jupyter nbextension enable --py gmaps
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


Figure(layout=FigureLayout(height='420px'))